# Lógica Computacional
João Carlos Viana Pereira Marques A84684 <br>
Saimon Alves Costa Sousa A76575



### **Trabalho Prático 1**



### Exercício 1


Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:
1. Cada reunião ocupa uma sala (enumeradas $1...S,$) durante um “slot” $(\text{tempo},\text{dia})$. Assume-se os dias enumerados $1..D$ e, em cada dia, os tempos enumerados $1..T$.
2. Cada reunião tem associado um projeto (enumerados $1..P$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $1..C$.
3. Cada projeto tem associado um conjunto de colaboradores, dos quais um é o líder. Cada projeto realiza um dado número de reuniões semanais. São “inputs” do problema o conjunto de colaboradores de cada projeto, o seu líder e o número de reuniões semanais.
4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de $50%$ do total de colaboradores do projeto. A disponibilidade de cada participante, incluindo o lider, é um conjunto de “slots” (“inputs” do problema).

 




**Análise do Problema**

Este é um problema de alocação.
Teremos a variavel S, D, T, P e C, que representam respetivamente, o número da salas disponíveis na StartUp, o número  dias de trabalho, o número de tempos por dia, o número do projeto e o número de colaboradores.

Utilizando as variáveis defenidas em cima
podemos criar uma matriz de alocação com o seguinte significado:

$$
X_{P,C,S,D,T} == 1 \quad \text{se e só se} \quad \text{o projeto $p$ é alocado ao colaborador $c$ na sala $s$ no dia $d$ no tempo $t$}
$$

Agora vamos enumerar as limitações que acho-mos pretinentes para a resolução do problema:
1. Cada sala tem no maximo 1 projeto de cada vez
$$
\forall_{s}\forall_{d}\forall_{t} \quad \sum_{p < P} {X_{p,c,s,d,t}} \le 1
$$ 
2. Cada projeto tem no máximo uma sala de cada vez
$$
\forall_{p}\forall_{d}\forall_{t} \quad \sum_{s < S} {X_{p,c,s,d,t}} \le 1
$$
3. Cada colaborador só pode estar no máximo num projeto em cada tempo dia
$$
\forall_{c}\forall_{s}\forall_{d}\forall_{t} \quad \sum_{p < P} {X_{p,c,s,d,t}} \le 1
$$

Implementões obrigatórias:

4. Cada projeto tem de realizar um N número de reúnioes semanais
$$
\forall_{p} \quad \sum_{s < S, d < D, t < T} {X_{p,c,s,d,t}} == N
$$
5. O líder tem de particpar em todas as reuniões do seu projeto
$$
\forall_{p} \sum_{s < S,t < T, d < D} {X_{p,l,s,d,t}} == N
$$


In [4]:
from ortools.linear_solver import pywraplp

In [8]:
def horario (S,D,T,P,C):
    
    horario = pywraplp.Solver.CreateSolver('SCIP')
    
    x = {}
    for s in range (S):
        for d in range (D):
            for t in range (T):
                for p in range (P):
                        x[s,d,t,p] = horario.BoolVar("x[%i,%i,%i,%i]" % (s,d,t,p)
                                                     
#cada sala tem 1 projeto de cada vez
        for s in range(S):
        for d in range(D):
            for t in range(T):
                horario.Add(sum([x[p,c,s,d,t] for p in range(P)]) <= 1)


#cada colaborador tem uma reunião em cada (T,D)
    for c in range(C):
        for s in range(S):
            for d in range(D):
                for t in range(T):
                    horario.Add(sum([x[p,c,s,d,t] for p in range(P)]) <= 1)

### Exercício 2

Queremos construir um programa que inicialize um tabuleiro sudoku $N^2$ $\times$ $N^2$, para $N \in \{3,4,5,6\}$, e que preencha uma fração $\alpha$, para  $\alpha \in \{0.0, 0.2, 0.4, 0.6\}$  das $N^4$ casas respeitando a regra:
- Cada inteiro no intervalo de $1$ a $N^2$ ocorre uma e uma só vez em cada linha, coluna ou secção

In [2]:
from ortools.linear_solver import pywraplp
import random

In [3]:
#declarar o solver
solver = pywraplp.Solver.CreateSolver('SCIP')


def sudoku(n,a):
    n_=n**2
    numcasas=round(n**4 * a)

    #definir casas a serem preenchidas
    A=[]
    for i in range(numcasas):
        A.append((random.randint(0,n_-1), random.randint(0,n_-1)))

    #criar variáveis
    x={}
    rows=range(0,n_)
    cols=range(0,n_)    
    val=range(1,n_+1)
    for r in rows:
        for c in cols:
            for v in val:
                x[r,c,v] = solver.BoolVar("x[%i,%i,%i]" % (r,c,v))

    #restrição para que em cada célula exista apenas um número de 1 a N^2
    for r in rows:
        for c in cols:
            solver.Add(sum(x[r,c,v] for v in val) == 1)

    #restrição para cada linha conter todos os números de 1 a N^2
    for r in rows:
       for v in val:
           solver.Add(sum(x[r,c,v] for c in cols) == 1)
        
    #restrição para cada coluna conter todos os números de 1 a N^2
    for c in rows:
        for v in val:
            solver.Add(sum(x[r,c,v] for r in rows) == 1)

    #restrição para aplicar as regras anteriores a cada sub-tabuleiro NxN 
    a = []
    count = n
    while(count<n_):
        a.append(count)
        count+=n
    for i in a:
        for k in a:     
            for v in val:
                solver.Add(sum(x[r+i,c+k,v] for r in range(0,n) for c in range(0,n)) == 1)
    
    #imprimir grelha com uma fração das casas preenchidas
    status=solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print(n_,' x ', n_)
        for r in rows :
            print(end='\n')
            for c in cols :
                for v in val:
                    if round(x[r,c,v].solution_value()) ==1 and (r,c) in A:
                        print('[', v, ']', end=' ')
                    elif round(x[r,c,v].solution_value()==1 ) and (r,c) not in A:
                        print('[ ',' ]', end=' ')
    print('\n')
    
    #imprimir grelha preenchida na totalidade
    status=solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        for r in rows :
            print(end='\n')
            for c in cols :
                for v in val:
                    if round(x[r,c,v].solution_value()) == 1:
                        print('[', v, ']', end=' ')
    print('\n\n')

    


In [4]:
sudoku(3,0.0)
sudoku(3,0.2)
sudoku(3,0.4)
sudoku(3,0.6)

9  x  9

[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 


[ 2 ] [ 5 ] [ 8 ] [ 3 ] [ 6 ] [ 1 ] [ 7 ] [ 9 ] [ 4 ] 
[ 7 ] [ 3 ] [ 6 ] [ 5 ] [ 9 ] [ 4 ] [ 2 ] [ 8 ] [ 1 ] 
[ 4 ] [ 1 ] [ 9 ] [ 2 ] [ 8 ] [ 7 ] [ 5 ] [ 6 ] [ 3 ] 
[ 6 ] [ 4 ] [ 3 ] [ 9 ] [ 2 ] [ 8 ] [ 1 ] [ 5 ] [ 7 ] 
[ 9 ] [ 7 ] [ 2 ] [ 1 ] [ 4 ] [ 5 ] [ 6 ] [ 3 ] [ 8 ] 
[ 5 ] [ 8 ] [ 1 ] [ 7 ] [ 3 ] [ 6 ] [ 4 ] [ 2 ] [ 9 ] 
[ 1 ] [ 2 ] [ 7 ] [ 8 ] [ 5 ] [ 3 ] [ 9 ] [ 4 ] [ 6 ] 
[ 8 ] [ 9 ] [ 4 ] [ 6 ] [ 1 ] [ 2 ] [ 3 ] [ 7 ] [ 5 ] 
[ 3 ] [ 6 ] [ 5 ] [ 4 ] [ 7 ] [ 9 ] [ 8 ] [ 1 ] [ 2 ] 

In [5]:
sudoku(4,0.0)

16  x  16

[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [

In [6]:
sudoku(4,0.2)

16  x  16

[   ] [   ] [   ] [ 11 ] [ 16 ] [   ] [ 14 ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [ 2 ] [   ] [ 13 ] [ 12 ] [   ] [   ] [ 16 ] [   ] [   ] 
[ 7 ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [ 10 ] [   ] [   ] [   ] [   ] [ 9 ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [ 6 ] [   ] [   ] [   ] [ 1 ] [   ] 
[   ] [   ] [   ] [   ] [ 2 ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [ 14 ] [   ] [ 10 ] [ 13 ] [   ] [   ] [ 1 ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [ 15 ] [   ] [   ] [ 2 ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [ 10 ] [   ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [ 16 ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [ 12 ] [ 7 ] [   ] [   ] [   ] [ 5 ] [   ] [ 9 ] [   ] [   ] [   ] [   ] [   ] [   ] 
[   ]

In [7]:
sudoku(4,0.4)

16  x  16

[   ] [   ] [ 10 ] [   ] [ 8 ] [   ] [ 6 ] [   ] [   ] [   ] [   ] [ 12 ] [   ] [   ] [   ] [   ] 
[   ] [ 9 ] [ 11 ] [ 8 ] [   ] [ 7 ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] 
[ 14 ] [   ] [ 6 ] [   ] [ 16 ] [   ] [ 10 ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [ 1 ] 
[   ] [ 2 ] [   ] [   ] [   ] [   ] [   ] [ 3 ] [   ] [ 13 ] [   ] [   ] [   ] [ 9 ] [ 8 ] [   ] 
[ 16 ] [   ] [ 13 ] [ 5 ] [ 2 ] [   ] [   ] [   ] [   ] [   ] [ 1 ] [   ] [   ] [   ] [   ] [   ] 
[   ] [   ] [ 12 ] [   ] [   ] [ 1 ] [   ] [ 13 ] [   ] [ 16 ] [   ] [ 9 ] [ 15 ] [   ] [ 2 ] [   ] 
[   ] [   ] [ 8 ] [   ] [   ] [   ] [   ] [   ] [ 11 ] [   ] [   ] [ 2 ] [ 9 ] [ 16 ] [   ] [ 3 ] 
[   ] [   ] [   ] [   ] [   ] [   ] [   ] [   ] [ 12 ] [   ] [   ] [ 7 ] [   ] [   ] [ 13 ] [ 11 ] 
[ 6 ] [ 1 ] [ 16 ] [ 4 ] [   ] [   ] [   ] [   ] [   ] [ 12 ] [   ] [   ] [   ] [ 2 ] [   ] [   ] 
[   ] [ 5 ] [   ] [   ] [   ] [ 15 ] [   ] [   ] [ 7 ] [   ] [   ] [   ] [   ] [   ] [ 16 ] [   

In [8]:
sudoku(4,0.6)

16  x  16

[   ] [   ] [ 15 ] [ 11 ] [ 13 ] [   ] [   ] [ 10 ] [   ] [   ] [ 6 ] [ 9 ] [   ] [   ] [   ] [   ] 
[ 13 ] [   ] [   ] [ 9 ] [   ] [   ] [   ] [ 8 ] [ 14 ] [ 15 ] [ 10 ] [   ] [ 7 ] [   ] [ 16 ] [   ] 
[ 12 ] [   ] [ 8 ] [ 5 ] [   ] [   ] [ 6 ] [   ] [ 7 ] [   ] [   ] [ 13 ] [   ] [   ] [   ] [ 15 ] 
[ 16 ] [   ] [ 10 ] [   ] [   ] [ 5 ] [   ] [ 4 ] [ 3 ] [   ] [ 8 ] [ 11 ] [   ] [   ] [ 14 ] [   ] 
[ 10 ] [   ] [ 5 ] [   ] [   ] [ 15 ] [   ] [   ] [   ] [   ] [ 4 ] [   ] [   ] [   ] [   ] [ 11 ] 
[   ] [   ] [ 11 ] [ 3 ] [   ] [   ] [ 7 ] [   ] [   ] [   ] [ 15 ] [ 10 ] [ 13 ] [   ] [ 1 ] [   ] 
[   ] [   ] [ 6 ] [ 4 ] [ 8 ] [ 13 ] [   ] [   ] [   ] [   ] [ 7 ] [ 3 ] [ 16 ] [   ] [ 2 ] [   ] 
[ 7 ] [   ] [ 16 ] [   ] [   ] [ 10 ] [   ] [   ] [   ] [   ] [   ] [ 8 ] [   ] [ 15 ] [   ] [   ] 
[   ] [   ] [   ] [ 16 ] [   ] [   ] [   ] [ 13 ] [   ] [   ] [   ] [   ] [ 3 ] [ 14 ] [   ] [   ] 
[   ] [   ] [   ] [   ] [ 1 ] [   ] [   ] [   ] [ 15 ] [ 3 ] [   ] [ 6 ] [ 9 ] [   ] 

In [ ]:
sudoku(5,0.0)
sudoku(5,0.2)
sudoku(5,0.4)
sudoku(5,0.6)

In [ ]:
sudoku(6,0.0)
sudoku(6,0.2)
sudoku(6,0.4)
sudoku(6,0.6)

Como podemos observar os tempos de execução tornam-se substancialmente mais elevados a medida que aumentamos o tamanho do tabuleiro e a fração de casas a serem preenchidas, isto deve-se ao aumento do número variáveis que resultará num aumento no numero de operações